# Seattle Collisions - Deployment

The final chapter here is deployment and the data product.  The end goal is to have this hosted in nbviewer as a dashboard.

In [29]:
from sklearn.neighbors import BallTree
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from math import sqrt
import pandas as pd
import types
import itertools
import numpy as np
from datetime import datetime, timedelta, timezone 
import time

import os
import folium
import folium.plugins

In [121]:
df = pd.read_csv('Seattle_Collisions_Final.csv', low_memory = False, parse_dates=True, index_col=0)
#print('File downloaded')
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])

In [119]:
def nearest_neighbors(df_source, nearest_in_miles = 0.01, quantile = .9, minimum_neighbors = 0):
    columns = list('YX')
    coordinates = df_source.loc[:,columns].values
    radius = nearest_in_miles / 3959.87433
    bt = BallTree(np.radians(coordinates), metric='haversine')
    neighbors = bt.query_radius(np.radians(coordinates), r=radius, count_only=True, return_distance=False)
    if np.quantile(neighbors, quantile, interpolation='nearest') > minimum_neighbors:
        return df_source.iloc[np.where(neighbors >= np.quantile(neighbors, quantile, interpolation='nearest'))[0]]
    else:
        if np.max(neighbors) > 10:
            return df_source.iloc[np.where(neighbors >= np.quantile(neighbors, quantile, interpolation='nearest'))[0]]

    return pd.DataFrame()

In [63]:
week = [0,1,2,3,4,5,6]
workweek = [0,1,2,3,4]
weekend = [5,6]
monday = [0]
tuesday = [1]
wednesday = [2]
thursday = [3]
friday = [4]
saturday = [5]
sunday = [6]

monday_color = 'red'
tuesday_color = 'yellow'
wednesday_color = 'orange'
thursday_color = 'blue'
friday_color = 'green'
saturday_color = 'brown'
sunday_color = 'purple'

In [105]:
def plot_map(marker_cluster, days = [0,1,2,3,4,5,6], reduce = True, start_date = '2015-12-31', stop_date = '2020-01-01',\
             with_rain = False, cluster_color = 'red', regional=True, quantile=0.9):
    if with_rain == False:
        query_filter = "INCDTTM > @start_date & INCDTTM < @stop_date & HITPARKEDCAR == 0 & PRECIPITATION == 0 & WEEKDAY in @days"
    else:
        query_filter = "INCDTTM > @start_date & INCDTTM < @stop_date & HITPARKEDCAR == 0 & PRECIPITATION > 0 & WEEKDAY in @days"

    if regional == True:
        df_filtered = df.query(query_filter).groupby('MCCP')
        for name, group in df_filtered:
            if reduce == True:
                group = nearest_neighbors(group, quantile=quantile)
            for index, row in group.iterrows():
                folium.CircleMarker(location=[row['Y'],row['X']],
                                    radius= 10,
                                    color=cluster_color,
                                    fill_color='orange',
                                    fill_opacity=0.5,
                                    fill=True).add_to(marker_cluster)
    else:
        df_filtered = df.query(query_filter)
        if reduce == True:
            df_filtered = nearest_neighbors(df_filtered, quantile=quantile)
        for index, row in df_filtered.iterrows():
            folium.CircleMarker(location=[row['Y'],row['X']],
                                radius= 10,
                                color=cluster_color,
                                fill_color='orange',
                                fill_opacity=0.5,
                                fill=True).add_to(marker_cluster)

In [111]:
def display_week_view(m, rain_flag=False):
    fg_1a = folium.FeatureGroup(name='Week 1', show=True).add_to(m.m1)
    fg_2a = folium.FeatureGroup(name='Week 2', show=True).add_to(m.m2)

    mc_1a = folium.plugins.MarkerCluster(name="Week 1")
    plot_map(mc_1a, week, with_rain=rain_flag, cluster_color=monday_color, regional=False, quantile=.99)
    mc_1a.add_to(fg_1a)

    mc_2a = folium.plugins.MarkerCluster(name="Week 2")
    plot_map(mc_2a, week, with_rain=rain_flag, cluster_color=tuesday_color, quantile=.99)
    mc_2a.add_to(fg_2a)
    folium.LayerControl().add_to(m)
    return

In [87]:
def display_workweek_weekend_view(m, rain_flag=False):
    fg_1a = folium.FeatureGroup(name='Workweek', show=True).add_to(m.m1)
    fg_2a = folium.FeatureGroup(name='Weekend', show=True).add_to(m.m2)

    mc_1a = folium.plugins.MarkerCluster(name="Workweek")
    plot_map(mc_1a, workweek, with_rain=rain_flag, cluster_color=monday_color)
    mc_1a.add_to(fg_1a)

    mc_2a = folium.plugins.MarkerCluster(name="Weekend")
    plot_map(mc_2a, weekend, with_rain=rain_flag, cluster_color=tuesday_color)
    mc_2a.add_to(fg_2a)
    folium.LayerControl().add_to(m)
    return

In [88]:
def display_workweek_view(m, rain_flag=False):
    fg_1a = folium.FeatureGroup(name='Workweek', show=True).add_to(m.m1)
    fg_2a = folium.FeatureGroup(name='Workweek', show=True).add_to(m.m2)

    mc_1a = folium.plugins.MarkerCluster(name="Workweek")
    plot_map(mc_1a, workweek, with_rain=rain_flag, cluster_color=monday_color)
    mc_1a.add_to(fg_1a)

    mc_2a = folium.plugins.MarkerCluster(name="Workweek")
    plot_map(mc_2a, workweek, with_rain=rain_flag, cluster_color=monday_color)
    mc_2a.add_to(fg_2a)
    folium.LayerControl().add_to(m)
    return

In [65]:
def display_weekend_view(m, rain_flag=False):
    fg_1a = folium.FeatureGroup(name='Weekend', show=True).add_to(m.m1)
    fg_2a = folium.FeatureGroup(name='Weekend', show=True).add_to(m.m2)

    mc_1a = folium.plugins.MarkerCluster(name="Weekend")
    plot_map(mc_1a, weekend, with_rain=rain_flag, cluster_color=monday_color)
    mc_1a.add_to(fg_1a)

    mc_2a = folium.plugins.MarkerCluster(name="Weekend")
    plot_map(mc_2a, weekend, with_rain=rain_flag, cluster_color=monday_color)
    mc_2a.add_to(fg_2a)
    folium.LayerControl().add_to(m)
    return

In [56]:
def display_daily_view(m, rain_flag=False):
    fg_1a = folium.FeatureGroup(name='Monday', show=True).add_to(m.m1)
    fg_1b = folium.FeatureGroup(name='Tuesday', show=False).add_to(m.m1)
    fg_1c = folium.FeatureGroup(name='Wednesday', show=False).add_to(m.m1)
    fg_1d = folium.FeatureGroup(name='Thursday', show=False).add_to(m.m1)
    fg_1e = folium.FeatureGroup(name='Friday', show=False).add_to(m.m1)
    fg_1f = folium.FeatureGroup(name='Saturday', show=False).add_to(m.m1)
    fg_1g = folium.FeatureGroup(name='Sunday', show=False).add_to(m.m1)

    fg_2a = folium.FeatureGroup(name='Monday', show=True).add_to(m.m2)
    fg_2b = folium.FeatureGroup(name='Tuesday', show=False).add_to(m.m2)
    fg_2c = folium.FeatureGroup(name='Wednesday', show=False).add_to(m.m2)
    fg_2d = folium.FeatureGroup(name='Thursday', show=False).add_to(m.m2)
    fg_2e = folium.FeatureGroup(name='Friday', show=False).add_to(m.m2)
    fg_2f = folium.FeatureGroup(name='Saturday', show=False).add_to(m.m2)
    fg_2g = folium.FeatureGroup(name='Sunday', show=False).add_to(m.m2)

    mc_1a = folium.plugins.MarkerCluster(name="Monday")
    plot_map(mc_1a, monday, with_rain=rain_flag, cluster_color=monday_color)
    mc_1b = folium.plugins.MarkerCluster(name="Tuesday")
    plot_map(mc_1b, tuesday, with_rain=rain_flag, cluster_color=tuesday_color)
    mc_1c = folium.plugins.MarkerCluster(name="Wednesday")
    plot_map(mc_1c, wednesday, with_rain=rain_flag, cluster_color=wednesday_color)
    mc_1d = folium.plugins.MarkerCluster(name="Thursday")
    plot_map(mc_1d, thursday, with_rain=rain_flag, cluster_color=thursday_color)
    mc_1e = folium.plugins.MarkerCluster(name="Friday")
    plot_map(mc_1e, friday, with_rain=rain_flag, cluster_color=friday_color)
    mc_1f = folium.plugins.MarkerCluster(name="Saturday")
    plot_map(mc_1f, saturday, with_rain=rain_flag, cluster_color=saturday_color)
    mc_1g = folium.plugins.MarkerCluster(name="Sunday")
    plot_map(mc_1g, sunday, with_rain=rain_flag, cluster_color=sunday_color)

    mc_1a.add_to(fg_1a)
    mc_1b.add_to(fg_1b)
    mc_1c.add_to(fg_1c)
    mc_1d.add_to(fg_1d)
    mc_1e.add_to(fg_1e)
    mc_1f.add_to(fg_1f)
    mc_1g.add_to(fg_1g)

    mc_2a = folium.plugins.MarkerCluster(name="Monday")
    plot_map(mc_2a, monday, with_rain=rain_flag, cluster_color=monday_color)
    mc_2b = folium.plugins.MarkerCluster(name="Tuesday")
    plot_map(mc_2b, tuesday, with_rain=rain_flag, cluster_color=tuesday_color)
    mc_2c = folium.plugins.MarkerCluster(name="Wednesday")
    plot_map(mc_2c, wednesday, with_rain=rain_flag, cluster_color=wednesday_color)
    mc_2d = folium.plugins.MarkerCluster(name="Thursday")
    plot_map(mc_2d, thursday, with_rain=rain_flag, cluster_color=thursday_color)
    mc_2e = folium.plugins.MarkerCluster(name="Friday")
    plot_map(mc_2e, friday, with_rain=rain_flag, cluster_color=friday_color)
    mc_2f = folium.plugins.MarkerCluster(name="Saturday")
    plot_map(mc_2f, saturday, with_rain=rain_flag, cluster_color=saturday_color)
    mc_2g = folium.plugins.MarkerCluster(name="Sunday")
    plot_map(mc_2g, sunday, with_rain=rain_flag, cluster_color=sunday_color)

    mc_2a.add_to(fg_2a)
    mc_2b.add_to(fg_2b)
    mc_2c.add_to(fg_2c)
    mc_2d.add_to(fg_2d)
    mc_2e.add_to(fg_2e)
    mc_2f.add_to(fg_2f)
    mc_2g.add_to(fg_2g)

    folium.LayerControl().add_to(m)

    return

In [ ]:
m = folium.plugins.DualMap(location=(47.6062, -122.3321), zoom_start=12, control_scale=True)
#display_daily_view(m, rain_flag=False)
#display_workweek_view(m, rain_flag=True)
#display_workweek_weekend_view(m, rain_flag=False)
display_week_view(m, rain_flag=False)
m